# Домашнее задание 3

## Сбор данных из интернета и генерация текстов

В этом домашнем задании вам предстоит: 
1. выбрать свой источник структурированных данных (прогноз погоды, курс акций, счет спортивных мероприятий, или что-нибудь еще) 
2. научиться извлекать данные из с выбранной веб-страницы
3. научиться генерировать текст по шаблонам

## Часть 1

В качестве источника данных используется сайт МФТИ https://mipt.ru/, извлекается информация о сотрудниках ректората (ФИО, должность и контактная информация).

In [1]:
import urllib.request 
import pandas as pd
import re

from nltk import tokenize
from nltk.corpus import stopwords
from pymystem3 import Mystem

from scrapy.selector import Selector
from scrapy.http import HtmlResponse
from tqdm import tqdm

In [2]:
link = urllib.request.urlopen('https://mipt.ru/persons/rectorat/')
 
lines = []
for line in link.readlines():
    #взяли все строки с сотрудниками
    if line.find(b'<h3>') != -1 and line.find(b'<a class') != -1 and line.find(b'href') != -1:
        lines.append(line)
link.close()

#Переводим bytes в str
for i in range(len(lines)):
    lines[i] = lines[i].decode('utf-8')
    
for i in range(len(lines)):
    #Убираем все лишнее
    lines[i] = lines[i].replace('\t\t\t<div class="name_pers">', '')
    lines[i] = lines[i].replace('<h3>', '')
    lines[i] = lines[i].replace('<a class="name_pers-headlink" href="', 'https://mipt.ru')
    lines[i] = lines[i].replace('">', ' ')
    lines[i] = lines[i].replace('</a></h3></div>\n', '')

#Фио сотрудников
names = []
#URL`ы с их контактными данными
urls = []
for line in lines:
    words = line.split(' ')
    urls.append(words[0])
    name = words[1] + ' ' + words[2] + ' ' + words[3]
    names.append(name)

In [3]:
#Ученая степень
degree = []
for url in urls:
    link = urllib.request.urlopen(url)
    for line in link.readlines():
        if line.find(b'<div class="person_main_academic_degree') == -1: #and 
            if line.find(b'<p>') != -1 and line.find(b'<em>') != -1:
                degree.append(line)
    link.close()
    
for i in range(len(degree)):
    degree[i] = degree[i].decode('utf-8')
    
for i in range(len(degree)):
    degree[i] = degree[i].replace('\t\t\t\t\t\t<p><em></em></p>\n', 'Нет информации')
    degree[i] = degree[i].replace('\t\t\t\t\t\t<p><em>', '')
    degree[i] = degree[i].replace('</em></p>\n', '')
    

In [4]:
#Контакты
contact = []
for url in urls:
    link = urllib.request.urlopen(url)
    for line in link.readlines():
        if line.find(b'<div class="person_main') == -1: 
            if line.find(b'<div class="person_main_contact">') == -1 and line.find(b'<p>') != -1 and line.find(b'<a href="mailto:') != -1:
                contact.append(line)
    
    link.close()

for i in range(len(contact)):
    contact[i] = contact[i].decode('utf-8')
for i in range(len(contact)):    
    contact[i] = contact[i].replace('\t\t\t            \t\t\t\t<p>', '')
    contact[i] = contact[i].replace('<a href="mailto:rector@mipt.ru">', '')
    contact[i] = contact[i].replace('</a></p>\n', '')
    contact[i] = contact[i].replace('          <p><a href="', '')
    contact[i] = contact[i].replace('+74954084800" target="_blank">', ' ')
    contact[i] = contact[i].replace('</a><br> <a href="mailto:pk@mipt.ru">', ' Эл. почта: ')
    contact[i] = contact[i].replace('                            <p>', '')
    contact[i] = contact[i].replace('\t\t\t            \t\t\t\t<p>', '')
    contact[i] = contact[i].replace('<a href="mailto:bagan.va@mipt.ru">', '')
    contact[i] = contact[i].replace('<a href="mailto:ffke@mipt.ru">', '')    
    contact[i] = contact[i].replace('\t\t\t\t\t\t\t<p>', '')
    contact[i] = contact[i].replace('\t\t\t            \t\t\t\t<p>', '')
    contact[i] = contact[i].replace('<a href="mailto:evgeny.evseev@gmail.com">', '')
    contact[i] = contact[i].replace('</a></p>\n', '')   
    contact[i] = contact[i].replace('          <p><a href="', '') 
    contact[i] = contact[i].replace('</a><br> <a href="mailto:pk@mipt.ru">', '') 
    contact[i] = contact[i].replace('\t\t\t            \t\t\t\t<p>', '')
    contact[i] = contact[i].replace('<a href="mailto:evgeny.evseev@gmail.com">', '')
    contact[i] = contact[i].replace('</a></p>\n', '')
    contact[i] = contact[i].replace('\t\t\t\t\t\t\t<p>', '')
    contact[i] = contact[i].replace('          <p><a href="', '')
    contact[i] = contact[i].replace('+74954084800" target="_blank">', ' ')
    contact[i] = contact[i].replace('</a><br> <a href="mailto:pk@mipt.ru">', ' ')
    contact[i] = contact[i].replace('+74954084800" target="_blank">', ' ')
    contact[i] = contact[i].replace('<a href="mailto:evseev.eg@mipt.ru">', ' ')
    contact[i] = contact[i].replace('<a href="mailto:zubtsov@phystech.edu">', ' ')
    contact[i] = contact[i].replace('<a href="mailto:zubtsov.da@mipt.ru">', ' ')
    contact[i] = contact[i].replace('+74954084800" target="_blank">', ' ') 
    contact[i] = contact[i].replace('tel', 'Тел') 

In [5]:
for i in range(len(names)):
    print(names[i], ':', degree[i], contact[i])

Кудрявцев Николай Николаевич : Доктор физико-математических наук, член-корреспондент РАН Эл. почта: rector@mipt.ru
Воронов Артем Анатольевич : Кандидат физико-математических наук Тел: +7 (495) 408-48-00 Эл. почта: pk@mipt.ru
Гаричев Сергей Николаевич : Доктор технических наук Тел: +7 (495) 408-48-00 Эл. почта: pk@mipt.ru
Анохова Елена Владимировна : кандидат экономических наук Тел: +7 (495) 408-48-00 Эл. почта: pk@mipt.ru
Ланчаков Александр Борисович : Нет информации Тел: +7 (495) 408-48-00 Эл. почта: pk@mipt.ru
Зайцев Константин Кириллович : Нет информации Тел: +7 (495) 408-48-00 Эл. почта: pk@mipt.ru
Петушков Владимир Сергеевич : Нет информации Тел: +7 (495) 408-48-00 Эл. почта: pk@mipt.ru
Дворкович Александр Викторович : Доктор технических наук, член-корр. РАН Тел: +7 (495) 408-48-00 Эл. почта: pk@mipt.ru
Киселев Валерий Валерьевич : Доктор физико-математических наук, доцент Тел: +7 (495) 408-48-00 Эл. почта: pk@mipt.ru
Негодяев Сергей Серафимович : Кандидат технических наук Тел: +7

## Часть 2
Запрограммируйте шаблон, описывающие температуру, уровень осадков и скорость ветра сегодня. Например,
"Температура сегодня составляет X градусов", где Х – излеченное выше число. Сгенерируйте таким образом описание погоды сегодня. Учтите, величина X влияет на грамматическое число, использованное в единицах измерения ("двадцать градусов", но "двадцать один градус"). Используйте бибилиотеку utg или дополнительные функции pymorphy2

In [6]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
butyavka = morph.parse('бутявка')[0] 
print(butyavka.inflect({'gent'}).word) #нет кого? родительный падеж
print(butyavka.make_agree_with_number(5).word) #согласование с числительным 5

бутявки
бутявок


In [7]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
for i in range(len(names)):
    familiya = morph.parse(names[i].split()[0])[0]  
    imya =  morph.parse(names[i].split()[1])[0]
    otchestvo =  morph.parse(names[i].split()[2])[0]
    print(familiya.inflect({'gent'}).word, imya.inflect({'gent'}).word, otchestvo.inflect({'gent'}).word) #нет кого? родительный падеж
    #print(butyavka.make_agree_with_number(5).word) #согласование с числительным 5

кудрявцева николая николаевича
воронова артёма анатолиевича
гаричева сергея николаевича
анохова елены владимировны
ланчаков александра борисовича
зайцева константина кирилловича
петушков владимира сергеевича
дворковича александра викторовича
киселева валерия валериевича
негодяев сергея серафимовича
иванова виктора владимировича
райгородского андрея михайловича
назарова виктора борисовича
багана виталия анатолиевича
евсеева евгения григориевича
зубцова дмитрия александровича
скалько юрия ивановича


In [9]:
import pymorphy2

morph = pymorphy2.MorphAnalyzer()

def generate_forms(word):
    p = morph.parse(word)[0]
    res = []
    lexems = p.lexeme
    if 'NOUN' in p.tag:
        for i in range(len(lexems)):
            res.append(lexems[i][4][0][1])
        #print(res)
        if 'femn' in p.tag:
            res1 = res[:5]
            res1.append(res[6])
            res1.extend(res[7:])
            res1.extend(res1[6:])
            return(res1)
        else:
            res.extend(res[6:])
            return(res)
    if 'VERB' in p.tag or 'INFN' in p.tag:
        for i in [0,1,3,2,4]:
            res.append(lexems[i][4][0][1])
        return(res)
    if 'ADJF' in p.tag:
        for i in range(18):
            res.append(lexems[i][4][0][1])
        return(res)

In [11]:
from utg import relations as r
from utg import logic
from utg import data
from utg import dictionary
from utg import words
from utg import templates
from utg import constructors

# описываем существительное для словаря
noun_word = words.Word(type=r.WORD_TYPE.NOUN,
                        forms=generate_forms('открытка'),
                        properties=words.Properties(r.ANIMALITY.INANIMATE, r.GENDER.FEMININE)) 

# описываем глагол для словаря
action_word = words.Word(type=r.WORD_TYPE.VERB,
                         forms=generate_forms('вручил') + [u''] * (len(data.WORDS_CACHES[r.WORD_TYPE.VERB]) - 5),
                         properties=words.Properties(r.ASPECT.PERFECTIVE, r.VOICE.DIRECT) )


# создаём словарь для использования в шаблонах
test_dictionary = dictionary.Dictionary(words=[noun_word, action_word])

# создаём шаблон
template = templates.Template()

# externals — внешние переменные, не обязаны быть в словаре
template.parse(u'[Obj|мн] [вручили|obj|прш] [subj|дт] [subj_1|дт] [red|вн|мр] [card|вн].', externals=('subj', 'subj_1', 'obj', 'card', 'red'))

for i in range(len(names)):
    imya =  morph.parse(names[i].split()[1])[0]
    otchestvo =  morph.parse(names[i].split()[2])[0]
# описываем внешние переменные
    subj = words.WordForm(words.Word(type=r.WORD_TYPE.NOUN,
                                         forms=generate_forms(imya[0])[:18],
                                         properties=words.Properties(r.ANIMALITY.ANIMATE)))
    
    subj_1 = words.WordForm(words.Word(type=r.WORD_TYPE.NOUN,
                                         forms=generate_forms(otchestvo[0])[12:30],
                                         properties=words.Properties(r.ANIMALITY.ANIMATE)))

    obj = words.WordForm(words.Word(type=r.WORD_TYPE.NOUN,
                                        forms=generate_forms('студент'),
                                         properties=words.Properties(r.ANIMALITY.ANIMATE, r.GENDER.MASCULINE)))
    card = words.WordForm(words.Word(type=r.WORD_TYPE.NOUN,
                                        forms=generate_forms('открытка'),
                                         properties=words.Properties(r.ANIMALITY.ANIMATE, r.GENDER.FEMININE)))

    red = words.WordForm(words.Word(type=r.WORD_TYPE.ADJECTIVE,
                                        forms=generate_forms('прощальный')[8:] + [u''] * (8+len(data.WORDS_CACHES[r.WORD_TYPE.ADJECTIVE]) - len(generate_forms('красная'))) ,
                                         properties=words.Properties(r.ANIMALITY.ANIMATE, r.GENDER.FEMININE, r.CASE.NOMINATIVE)))

        # осуществляем подстановку
    result = template.substitute(externals={'subj': subj,
                                            'subj_1': subj_1,
                                                'obj': obj,
                                               'card':card,
                                            'red':red},
                                     dictionary=test_dictionary)
    print(result)


Студенты вручили николаю николаевичу прощальную открытку.
Студенты вручили артёму анатолиевича прощальную открытку.
Студенты вручили сергею николаевичу прощальную открытку.
Студенты вручили елене владимировича прощальную открытку.
Студенты вручили александру борисовичу прощальную открытку.
Студенты вручили константину кирилловичу прощальную открытку.
Студенты вручили владимиру сергеевичу прощальную открытку.
Студенты вручили александру викторовичу прощальную открытку.
Студенты вручили валерию валериевича прощальную открытку.
Студенты вручили сергею серафимовичу прощальную открытку.
Студенты вручили виктору владимировичу прощальную открытку.
Студенты вручили андрею михайловичам прощальную открытку.
Студенты вручили виктору борисовичу прощальную открытку.
Студенты вручили виталию анатолиевича прощальную открытку.
Студенты вручили евгению григориевича прощальную открытку.
Студенты вручили дмитрию александровичу прощальную открытку.
Студенты вручили юрию ивановичу прощальную открытку.


## Часть 3 (необязательная)
Соберите данные не только за сегодняшний, но и за вчерашний день, и напишите шаблоны, которые сравнивают вчерашние и сегодняшние показатели (например, "сегодня похолодало на Х градусов по сравнению со вчера").